# Validating LLM Outputs with Pydantic

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/ShreyaR/guardrails/blob/main/docs/integrations/pydantic_validation.ipynb).

In this example, we will use Guardrails with Pydantic.

## Objective

We want to generate synthetic data that is consistent with a `Person` Pydantic BaseModel.

In [1]:
import guardrails as gd

from rich import print

## Step 1: Create the RAIL Spec

Ordinarily, we would create an RAIL spec in a separate file. For the purposes of this example, we will create the spec in this notebook as a string following the RAIL syntax. For more information on RAIL, see the [RAIL documentation](../rail/output.md).

Here, we define a Pydantic model for a `Person` with the following fields:

- `name`: a string  
- `age`: an integer  
- `zip_code`: a string zip code

and write very simple validators for the fields as an example. As a way to show how LLM reasking can be used to generate data that is consistent with the Pydantic model, we can define a validator that asks for a zip code in California (including being perversely opposed to the "90210" zip code). If this validator fails, the LLM will be sent the error message and will reask the question.

This Pydantic model could also be any model that you already have in your codebase, and just needs to be decorated with `@register_pydantic`.


To use this model in the `<output>` specification, we used the special
`pydantic` tag. This tag takes the name of the Pydantic model, as well as the
`on-fail-pydantic` attribute, which specifies what to do when the output
does not validate against the Pydantic model.

In [2]:
rail_str = """
<rail version="0.1">

<script language="python">
from guardrails.utils.pydantic_utils import register_pydantic
from pydantic import BaseModel, validator

@register_pydantic
class Person(BaseModel):
    '''
    Information about a person.

    Args:
        name (str): The name of the person.
        age (int): The age of the person.
        zip_code (str): The zip code of the person.
    '''
    name: str
    age: int
    zip_code: str

    @validator("zip_code")
    def zip_code_must_be_numeric(cls, v):
        if not v.isnumeric():
            raise ValueError("Zip code must be numeric.")
        return v

    @validator("age")
    def age_must_be_between_0_and_150(cls, v):
        if not 0 &lt;= v &lt;= 150:
            raise ValueError("Age must be between 0 and 150.")
        return v

    @validator("zip_code")
    def zip_code_in_california(cls, v):
        if not v.startswith("9"):
            raise ValueError("Zip code must be in California, and start with 9.")
        if v == "90210":
            raise ValueError("Zip code must not be Beverly Hills.")
        return v

</script>

<output>
    <list name="people" description="A list of 3 people.">
        <pydantic description="Information about a person." model="Person" on-fail-pydantic="reask" />
    </list>
</output>


<prompt>
Generate data for possible users in accordance with the specification below.

${xml_prefix_prompt}

${output_schema}

${complete_json_suffix_v2}</prompt>

</rail>
"""

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

In [3]:
guard = gd.Guard.from_rail_string(rail_str)

We see the prompt that will be sent to the LLM.

In [4]:
print(guard.base_prompt)

Generate data for possible users in accordance with the specification below.


Given below is XML that describes the information to extract from this document and the tags to extract it into.


<output>
    <list name="people" description="A list of 3 people.">
        <object description="Information about a person." pydantic="Person"><string name="name" description="The 
name of the person."/><integer name="age" description="The age of the person." 
format="age-must-be-between-0-and-150"/><string name="zip_code" description="The zip code of the person." 
format="zip-code-must-be-numeric; zip-code-in-california"/></object></list>
</output>


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <list name="people" description="A list of 3 people.">
        <object description="Information about a person." pydantic="Person"><string name="name" description="The 
name of the person."/><integer name="age" description="The age of the person." 
format="age-must-be-between-0-and-150"/><string name="zip_code" description="The zip code of the person." 
format="zip-code-must-be-numeric; zip-code-in-california"/></object></list>
</output>

ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{{'foo': 'example one'}}`
- `<list name='bar'><string format='upper-case' /></list>` => `{{"bar": ['STRING ONE', 'STRING TWO', etc.]}}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{{'baz': {{'foo': 'Some String', 'index': 1}}}}`

JSON Object:

!!! note
    Notice that the prompt replaces the `pydantic` tag with the schema, validator and type information from the Pydantic model. This e.g. tells the LLM that we want that `zip-code-must-be-numeric` and `zip-code-in-california`. Guardrails will even automatically read the docstrings from the Pydantic model and add them to the prompt!

## Step 3: Wrap the LLM API call with `Guard`

In [5]:
import openai

raw_llm_response, validated_response = guard(
    openai.Completion.create,
    engine="text-davinci-003",
    max_tokens=512,
    temperature=0.5,
    num_reasks=2,
)

/Users/krandiash/opt/anaconda3/envs/guardrails/lib/python3.9/site-packages/eliot/json.py:22: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if isinstance(o, (numpy.bool, numpy.bool_)):


In [6]:
print(validated_response)

{
    'people': [
        Person(name='John Doe', age=25, zip_code='90000'),
        Person(name='Jane Doe', age=30, zip_code='94105'),
        Person(name='John Smith', age=40, zip_code='90001')
    ]
}

The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary).

We can see that the output is a dictionary with the correct schema and contains a few `Person` objects!

We can even print out the logs of the most recent call. Notice that the first time the LLM actually returns a Beverly Hills zip code, the LLM is sent the error message and is reasked. The second time, the LLM returns a valid zip code and the output is returned.

In [8]:
print(guard.state.most_recent_call)

GuardHistory(
    history=[
        GuardLogs(
            prompt='\nGenerate data for possible users in accordance with the specification below.\n\n\nGiven below
is XML that describes the information to extract from this document and the tags to extract it 
into.\n\n\n<output>\n    <list name="people" description="A list of 3 people.">\n        <object 
description="Information about a person." pydantic="Person"><string name="name" description="The name of the 
person."/><integer name="age" description="The age of the person." format="age-must-be-between-0-and-150"/><string 
name="zip_code" description="The zip code of the person." format="zip-code-must-be-numeric; 
zip-code-in-california"/></object></list>\n</output>\n\n\nGiven below is XML that describes the information to 
extract from this document and the tags to extract it into.\n\n<output>\n    <list name="people" description="A 
list of 3 people.">\n        <object description="Information about a person." pydantic="Person"><string 
name="name" description="The name of the person."/><integer name="age" description="The age of the person." 
format="age-must-be-between-0-and-150"/><string name="zip_code" description="The zip code of the person." 
format="zip-code-must-be-numeric; zip-code-in-california"/></object></list>\n</output>\n\nONLY return a valid JSON 
object (no other text is necessary), where the key of the field in JSON is the `name` attribute of the 
corresponding XML, and the value is of the type specified by the corresponding XML\'s tag. The JSON MUST conform to
the XML format, including any types and format requests e.g. requests for lists, objects and specific types. Be 
correct and concise.\n\nHere are examples of simple (XML, JSON) pairs that show the expected behavior:\n- `<string 
name=\'foo\' format=\'two-words lower-case\' />` => `{\'foo\': \'example one\'}`\n- `<list name=\'bar\'><string 
format=\'upper-case\' /></list>` => `{"bar": [\'STRING ONE\', \'STRING TWO\', etc.]}`\n- `<object 
name=\'baz\'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{\'baz\': {\'foo\': \'Some String\', \'index\': 1}}`\n\nJSON Object:',
            output=' \n{\n    "people": [\n        {\n            "name": "John Doe",\n            "age": 25,\n    
"zip_code": "90210"\n        },\n        {\n            "name": "Jane Doe",\n            "age": 30,\n            
"zip_code": "94105"\n        },\n        {\n            "name": "John Smith",\n            "age": 40,\n            
"zip_code": "90001"\n        }\n    ]\n}',
            output_as_dict={
                'people': [
                    {'name': 'John Doe', 'age': 25, 'zip_code': '90210'},
                    {'name': 'Jane Doe', 'age': 30, 'zip_code': '94105'},
                    {'name': 'John Smith', 'age': 40, 'zip_code': '90001'}
                ]
            },
            validated_output={
                'people': [
                    {
                        'name': 'John Doe',
                        'age': 25,
                        'zip_code': ReAsk(
                            incorrect_value='90210',
                            error_message='Zip code must not be Beverly Hills.',
                            fix_value=None,
                            path=['people', 0]
                        )
                    },
                    Person(name='Jane Doe', age=30, zip_code='94105'),
                    Person(name='John Smith', age=40, zip_code='90001')
                ]
            },
            reasks=[
                ReAsk(
                    incorrect_value='90210',
                    error_message='Zip code must not be Beverly Hills.',
                    fix_value=None,
                    path=['people', 0]
                )
            ]
        ),
        GuardLogs(
            prompt='\nI was given the following JSON response, which had problems due to incorrect values.\n\n{\n  
"people": [\n    {\n      "name": "J